In [ ]:
#results = []

In [19]:
# this cell can be run multiple times to concatenate results

# models from OpenAI
import os, time
from openai import OpenAI
import base64
import pandas as pd

start = len(results)

# Classes to classify into
class_list = [
    'baby', 'bed', 'bicycle', 'chimpanzee', 'fox', 'leopard', 'man',
    'pickup_truck', 'plain', 'poppy', 'rocket', 'rose', 'snail',
    'sweet_pepper', 'table', 'telephone', 'wardrobe', 'whale', 'woman',
    'worm'
]

# Helper function to convert image to base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

def classify_image_with_retry(model, message, retries=3):

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENAI_KEY,
    )
    max_attempts = 5  # Number of retry attempts
    success = False
    attempt = 0
    while attempt < max_attempts and not success:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=message)
            answer = completion.choices[0].message.content.strip()
            success = True
            return answer

        except Exception as e:
            print(e)
            print(f"\t⏳ Rate limit hit for {model}. Waiting 60 seconds before retrying (Attempt {attempt+1}/{max_attempts})...")
            time.sleep(60)
            attempt += 1
    return None


models = [
    #"openai/gpt-4.1-nano",
    "mistralai/mistral-small-3.1-24b-instruct:free", 
    "qwen/qwen2.5-vl-72b-instruct:free",
    "google/gemma-3-27b-it:free",
]

root_dir = "/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/"
i = 0
MAX_IMAGES_PER_CLASS = 10

for class_name in os.listdir(root_dir):
    class_folder = os.path.join(root_dir, class_name)
    if class_name in {'bed', 'man', 'snail', 'wardrobe'}:
        continue
    if not os.path.isdir(class_folder):
        continue

    images = os.listdir(class_folder)[:MAX_IMAGES_PER_CLASS]
    for img_name in images:
        img_path = os.path.join(class_folder, img_name)
        try:
            base64_image = encode_image(img_path)

            message=[{
                "role": "user",
                "content": [{
                        "type": "text",
                        "text": f"This image belongs to one of the following classes:\n\n"
                                f"{', '.join(class_list)}.\n\n"
                                f"Please classify it by returning only the most appropriate class name from the list."
                    }, {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}",},
                    }]}]
            
            # Initialize row for this image
            row = {
                "image_path": img_path,
                "real_class": class_name
            }

            for model in models:
                pred = classify_image_with_retry(model, message)
                row[f"pred_{model}"] = pred if pred else "None"

            results.append(row)
    
            i += 1
            print(f"{i}. {row}")

        except Exception as e:
            print(f"Error with {img_path}: {e}")

# Convert results to DataFrame
df = pd.DataFrame(results)
df.to_csv("many_classification_results.csv", index=False)

print("Classification completed. Results saved to many_classification_results.csv")

1. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/fox/red_fox_s_000209.png', 'real_class': 'fox', 'pred_mistralai/mistral-small-3.1-24b-instruct:free': 'woman', 'pred_qwen/qwen2.5-vl-72b-instruct:free': 'fox', 'pred_google/gemma-3-27b-it:free': 'fox'}
2. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/fox/fox_s_000786.png', 'real_class': 'fox', 'pred_mistralai/mistral-small-3.1-24b-instruct:free': 'snail', 'pred_qwen/qwen2.5-vl-72b-instruct:free': 'snail', 'pred_google/gemma-3-27b-it:free': 'fox'}
3. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/fox/red_fox_s_000962.png', 'real_class': 'fox', 'pred_mistralai/mistral-small-3.1-24b-instruct:free': 'fox', 'pred_qwen/qwen2.5-vl-72b-instruct:free': 'fox', 'pred_google/gemma-3-27b-it:free': 'fox'}
4. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/fox/fox_s_000976.png', 'real_class': 'fox', 'pred_mistralai/mistral-small-3.1-24b-instruct:free': 

KeyboardInterrupt: 

In [20]:
print(len(results))

948


In [21]:
import pandas as pd

# Convert results to DataFrame
df = pd.DataFrame(results)
df.to_csv("many_classification_results.csv", index=False)

print("Classification completed. Results saved to many_classification_results.csv")

Classification completed. Results saved to many_classification_results.csv


In [22]:
df

,image_path,real_class,pred_mistralai/mistral-small-3.1-24b-instruct:free,pred_qwen/qwen2.5-vl-72b-instruct:free,pred_google/gemma-3-27b-it:free
0,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,woman,man,baby
1,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
2,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
3,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
4,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,woman
...,...,...,...,...,...
943,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,poppy,poppy,poppy
944,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,poppy,poppy,poppy
945,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,rose,poppy,poppy
946,/Users/jdieguez/Documents/master/TFM/code/cifa...,table,table,table,table


In [23]:
df_all = df

In [24]:
df_all

,image_path,real_class,pred_mistralai/mistral-small-3.1-24b-instruct:free,pred_qwen/qwen2.5-vl-72b-instruct:free,pred_google/gemma-3-27b-it:free
0,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,woman,man,baby
1,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
2,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
3,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,man
4,/Users/jdieguez/Documents/master/TFM/code/cifa...,man,man,man,woman
...,...,...,...,...,...
943,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,poppy,poppy,poppy
944,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,poppy,poppy,poppy
945,/Users/jdieguez/Documents/master/TFM/code/cifa...,poppy,rose,poppy,poppy
946,/Users/jdieguez/Documents/master/TFM/code/cifa...,table,table,table,table


In [25]:
df = df_all
df["pred"] = df["pred_mistralai/mistral-small-3.1-24b-instruct:free"].str.lower()
df = df[df["pred"] != "none"]
correct = (df["real_class"].str.lower() == df["pred"])
accuracy = correct.sum() / len(df)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.6702


In [26]:
df = df_all
df["pred"] = df["pred_qwen/qwen2.5-vl-72b-instruct:free"].str.lower()
df = df[df["pred"] != "none"]
correct = (df["real_class"].str.lower() == df["pred"])
accuracy = correct.sum() / len(df)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8624


In [27]:
df = df_all
df["pred"] = df["pred_google/gemma-3-27b-it:free"].str.lower()
df = df[df["pred"] != "none"]
correct = (df["real_class"].str.lower() == df["pred"])
accuracy = correct.sum() / len(df)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.8233


In [34]:
results_gpt = []

In [36]:
# this cell can be run multiple times to concatenate results

# models from OpenAI
import os, time
from openai import OpenAI
import base64
import pandas as pd

# Classes to classify into
class_list = [
    'baby', 'bed', 'bicycle', 'chimpanzee', 'fox', 'leopard', 'man',
    'pickup_truck', 'plain', 'poppy', 'rocket', 'rose', 'snail',
    'sweet_pepper', 'table', 'telephone', 'wardrobe', 'whale', 'woman',
    'worm'
]

# Helper function to convert image to base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

def classify_image_with_retry(model, message, retries=3):

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=OPENAI_KEY,
    )
    max_attempts = 5  # Number of retry attempts
    success = False
    attempt = 0
    while attempt < max_attempts and not success:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=message)
            answer = completion.choices[0].message.content.strip()
            success = True
            return answer

        except Exception as e:
            print(e)
            print(f"\t⏳ Rate limit hit for {model}. Waiting 60 seconds before retrying (Attempt {attempt+1}/{max_attempts})...")
            time.sleep(60)
            attempt += 1
    return None


models = [
    #"openai/gpt-4.1-nano",
    "openai/gpt-4o-mini",
    #"mistralai/mistral-small-3.1-24b-instruct:free", 
    #"qwen/qwen2.5-vl-72b-instruct:free",
    #"google/gemma-3-27b-it:free",
]

root_dir = "/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/"
i = 0
MAX_IMAGES_PER_CLASS = 10

for class_name in os.listdir(root_dir):
    class_folder = os.path.join(root_dir, class_name)
    if not os.path.isdir(class_folder):
        continue

    images = os.listdir(class_folder)[:MAX_IMAGES_PER_CLASS]
    for img_name in images:
        img_path = os.path.join(class_folder, img_name)
        try:
            base64_image = encode_image(img_path)

            message=[{
                "role": "user",
                "content": [{
                        "type": "text",
                        "text": f"This image belongs to one of the following classes:\n\n"
                                f"{', '.join(class_list)}.\n\n"
                                f"Please classify it by returning only the most appropriate class name from the list."
                    }, {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{base64_image}",},
                    }]}]
            
            # Initialize row for this image
            row = {
                "image_path": img_path,
                "real_class": class_name
            }

            for model in models:
                pred = classify_image_with_retry(model, message)
                row[f"pred_{model}"] = pred if pred else "None"

            results_gpt.append(row)
    
            i += 1
            print(f"{i}. {row}")

        except Exception as e:
            print(f"Error with {img_path}: {e}")

# Convert results to DataFrame
df = pd.DataFrame(results_gpt)
df.to_csv("gpt_classification_results.csv", index=False)

print("Classification completed. Results saved to gpt_classification_results.csv")

1. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/adam_s_002297.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
2. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/man_s_000089.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
3. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/adam_s_002095.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
4. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/abel_s_000720.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
5. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/abel_s_000091.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
6. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man/man_s_002305.png', 'real_class': 'man', 'pred_openai/gpt-4o-mini': 'man'}
7. {'image_path': '/Users/jdieguez/Documents/master/TFM/code/cifar-fs/test/man

In [37]:
df

,image_path,real_class,pred_openai/gpt-4o-mini
0,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,I don't know who they are.
1,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,chimpanzee
2,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,fox
3,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,fox
4,/Users/jdieguez/Documents/master/TFM/code/cifa...,fox,fox
...,...,...,...
355,/Users/jdieguez/Documents/master/TFM/code/cifa...,woman,woman
356,/Users/jdieguez/Documents/master/TFM/code/cifa...,woman,woman
357,/Users/jdieguez/Documents/master/TFM/code/cifa...,woman,woman
358,/Users/jdieguez/Documents/master/TFM/code/cifa...,woman,woman


In [38]:
df["pred"] = df["pred_openai/gpt-4o-mini"].str.lower()
df = df[df["pred"] != "none"]
correct = (df["real_class"].str.lower() == df["pred"])
accuracy = correct.sum() / len(df)

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.9639
